In [1]:
import pandas as pd
import numpy as np
import random
from matplotlib.pyplot import pie, axis, show
import seaborn as sns
import missingno as msno
from scipy import stats
import matplotlib.pyplot as plt
import yaml

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_selection import RFE, SelectKBest, f_regression, mutual_info_regression
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn import linear_model
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
import statsmodels.regression.linear_model as sm
from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, CompoundKernel
import sklearn_relief as sr
from skrebate import ReliefF
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import ElasticNet
import lightgbm as ltb
from sklearn.svm import SVR
from scipy.stats import ks_2samp
from tabulate import tabulate

from xgboost.sklearn import XGBRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.over_sampling import BorderlineSMOTE

from sklearn.multioutput import RegressorChain, MultiOutputRegressor

import torch
from torch.utils.data import Dataset, DataLoader
torch.manual_seed(42)

from helper import preprocess, countUsers, get_features_relieff, get_features_ref_single, get_features_ref,\
    get_features_kbest, outlier_detect, cross_val, get_scores

%matplotlib inline


In [2]:
df = pd.read_csv('../data/X_train.csv', sep = ',',decimal = '.', encoding = 'utf-8', engine ='python', index_col=0)

In [3]:
# Read common variables from a YAML file
with open('../../common_variables.yaml', 'r') as file:
    common_data = yaml.safe_load(file)

In [4]:
target_variable = 'hba1c_12m'
df, X_train, X_test, Y_train, Y_test, X, Y, scaler, df_missing_val, df_missing_val_original, df_original = preprocess(df, 0.25, target_variable)
df = df.drop(['ldl_12m', 'bmi_12m', 'hdl_12m'], axis=1)
X_train = X_train.drop(['ldl_12m', 'bmi_12m', 'hdl_12m'], axis=1)
X_test = X_test.drop(['ldl_12m', 'bmi_12m', 'hdl_12m'], axis=1)

Shape of data : (2660, 125)
Shape of data after excluding missing response: (2331, 125)
Shape of full data after selecting date range dates > 21 days (1752, 116)


In [5]:
df_missing_val

,id,init_year,drug_class,MD_RCT_mmol_mol,hba1c_bl_18m,hba1c_bl_6m,sp,ika,t2d_dur_y,P_Krea,...,dg132,dg133,n_of_dis,days_hba1c,days_bmi,days_hdl,days_ldl,ldl_12m,hdl_12m,bmi_12m
6673,0.005106,0.000,0.0,0.960794,0.161905,0.171875,1.0,0.500000,0.095238,0.107438,...,0.0,0.0,0.071429,1.0,0.152629,0.102099,0.14585,0.296296,0.228571,0.000245
8656,0.006635,0.875,1.0,0.411912,0.161905,0.046875,1.0,0.474359,0.119048,0.123967,...,0.0,0.0,0.214286,1.0,0.152629,0.102099,0.14585,0.296296,0.228571,0.000245
2857,0.002162,1.000,0.0,0.823574,0.161905,0.375000,0.0,0.512821,0.166667,0.061983,...,0.0,0.0,0.500000,1.0,0.289440,0.102099,0.14585,0.296296,0.228571,0.000152
12113,0.009255,0.875,1.0,0.882382,0.276190,0.046875,1.0,0.397436,0.166667,0.165289,...,0.0,0.0,0.214286,1.0,0.122274,0.102099,0.14585,0.296296,0.228571,0.000571
6305,0.004845,0.375,0.0,0.823574,0.123810,0.046875,0.0,0.794872,0.285714,0.173554,...,0.0,0.0,0.214286,1.0,0.152629,0.102099,0.14585,0.296296,0.228571,0.000245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7848,0.006048,0.875,0.0,0.823574,0.161905,0.171875,0.0,0.974359,0.000000,0.206612,...,0.0,0.0,0.285714,1.0,0.105173,0.102099,0.14585,0.296296,0.228571,0.000271
6216,0.004768,1.000,0.0,0.960794,0.257143,0.343750,1.0,0.769231,0.261905,0.334711,...,1.0,0.0,0.428571,1.0,0.147926,0.102099,0.14585,0.296296,0.228571,0.000215
8133,0.006267,1.000,1.0,1.000000,0.190476,0.000000,1.0,0.679487,0.142857,0.177686,...,0.0,0.0,0.428571,1.0,0.152629,0.102099,0.14585,0.296296,0.228571,0.000245
6160,0.004716,0.875,1.0,1.000000,0.285714,0.484375,1.0,0.538462,0.238095,0.276860,...,0.0,0.0,0.142857,1.0,1.213339,0.102099,0.14585,0.296296,0.228571,0.000216


In [6]:
# # train with whole dataset and test with drug class 2,3 and 4 data
is_train_with_all=False
if(is_train_with_all):
    combined_df = pd.concat([X_test, Y_test], axis=1)
    testdf = combined_df[(combined_df['drug_class'] == 0.25) | 
                         (combined_df['drug_class'] == 0.375) ]
    X_test = testdf.drop([response_variable], axis = 1)
    Y_test = testdf[response_variable]
    
X_test_original = X_test.copy()

In [7]:
""""
Use drug_class

2=GLP-1 analogues (A10BJ)
3=DPP-4 inhibitors (A10BH)
4=SGLT2 inhibitors (A10BK)
"""

if(is_train_with_all):
    sglt_val = 0.375
    dpp_val = 0.25
else:
    sglt_val = 1
    dpp_val = 0


X_test_ = pd.DataFrame(X_test)
X_train_ = pd.DataFrame(X_train)

X_train = X_train.drop(['init_year'], axis = 1)
X_test = X_test.drop(['init_year'], axis = 1)


print('==== sample count in preprocessed data =======')
print(' number of dpp4 : ', countUsers(3, df))
print(' number of sglt2 : ', countUsers(4, df))

print('==== sample count in training data =======')
print(' number of dpp4 : ', countUsers(dpp_val, X_train_))
print(' number of sglt2 : ', countUsers(sglt_val, X_train_))

print('==== sample count in testing data =======')
print(' number of dpp4 : ', countUsers(dpp_val, X_test_))
print(' number of sglt2 : ', countUsers(sglt_val, X_test_))

==== sample count in preprocessed data =======
 number of dpp4 :  679
 number of sglt2 :  1073
==== sample count in training data =======
 number of dpp4 :  516
 number of sglt2 :  798
==== sample count in testing data =======
 number of dpp4 :  163
 number of sglt2 :  275


In [8]:
# TODO FROM HERE

# feature selection
items = [
#     'sp',
#     'smoking',
    ]
k = 10 # Select top 25 features
    
random.seed(42)

#feats = get_features_ref_single(X_train, Y_train,20)
#feats = get_features_relieff(X_train, Y_train['hba1c_12m'] ,15)
#feats = get_features_kbest(X_train, Y_train,10)

# feats got from reliefF with 15
feats = ['hba1c_bl_6m', 'insulin', 'hba1c_bl_18m', 'sum_diab_drugs', 'MD_RCT_mmol_mol', 'drug_class', 'dpp4', 't2d_dur_y', 'gluk', 'concordant_dis', 'met_oad0', 'trigly', 'hyperten', 'ika']
print(feats)
selected_features=feats
        
X_train = X_train[selected_features]
X_test = X_test[selected_features]
number_of_features = len(selected_features)

['hba1c_bl_6m', 'insulin', 'hba1c_bl_18m', 'sum_diab_drugs', 'MD_RCT_mmol_mol', 'drug_class', 'dpp4', 't2d_dur_y', 'gluk', 'concordant_dis', 'met_oad0', 'trigly', 'hyperten', 'ika']


In [9]:
################# OUTLIER CODE ################
print('Shape of training data before removing outliers:', np.shape(X_train))
print('Shape of test data before removing outliers:', np.shape(X_test))
    
out_train, out_test = outlier_detect(X_train, Y_train, X_test, Y_test)
response_variable_list = [target_variable]

train_ = X_train.copy()
train_[response_variable_list] = Y_train.values
    
test_ = X_test.copy()
test_[response_variable_list] = Y_test.values
    
train_ = pd.DataFrame(train_.drop(out_train, axis = 0))
test_ = pd.DataFrame(test_.drop(out_test, axis = 0))
    
Y_train = train_[response_variable_list]
X_train = train_.drop(response_variable_list, axis=1)
    
Y_test = test_[response_variable_list]
X_test = test_.drop(response_variable_list, axis=1)
    
print('Shape of training data after removing outliers:', np.shape(X_train))
print('Shape of test data after removing outliers:', np.shape(X_test))

################

Shape of training data before removing outliers: (1314, 14)
Shape of test data before removing outliers: (438, 14)
Training set outliers: [340, 3662, 3693, 6014, 11741]
Testing set outliers: [11564]
Shape of training data after removing outliers: (1309, 14)
Shape of test data after removing outliers: (437, 14)


In [10]:
train = X_train.copy()
train[response_variable_list] = Y_train[response_variable_list].copy()


# Models

In [11]:
model = XGBRegressor(
    n_estimators=20, 
    eta=0.001, 
    subsample=0.2, 
    colsample_bytree=0.8,
    alpha=0.1,
    max_depth = 15,
    max_leaves = 10,
    learning_rate =0.1
)

#model = CatBoostRegressor(iterations=20,learning_rate=0.1, depth=6)

#model = RandomForestRegressor(n_estimators=150, max_depth=10, random_state=123)
model = MLPRegressor(random_state=123, max_iter=2000,hidden_layer_sizes = 16,learning_rate= 'adaptive')

model = cross_val(model, train , X_train, Y_train, response_variable_list)
model.fit(X_train, Y_train)
# make a prediction

yhat = model.predict(X_test)
# summarize prediction
print(yhat[1])
original_data_pred, model_results, model_results_drugs_ori, score_ori = get_scores(model, X_test, Y_test, X_train, Y_train)


/Users/anushaihalapathirana/anaconda3/envs/EUF/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/anushaihalapathirana/anaconda3/envs/EUF/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/anushaihalapathirana/anaconda3/envs/EUF/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/anushaihalapathirana/anaconda3/envs/EUF/lib/python3.11/site-packages/sklearn/neural_net

Cross validation variance 0.0034985521156825497
Cross validation mean score 0.41095921946435865
50.16428910131443
R2 score Training : 0.4533285725815641
R2 score Testing : 0.4009036844768954
RMSE (Target): 9.13239691384006


In [12]:

df_missing_val = df_missing_val[selected_features]
mv_pred_test_numpy = model.predict(df_missing_val)


In [13]:
len(mv_pred_test_numpy)

329

In [14]:
df_missing_val_original['hba1c_12m'] = mv_pred_test_numpy

In [15]:
df_missing_val_original['hba1c_12m']

6673     53.334371
8656     45.124345
2857     62.081110
12113    53.281130
6305     50.841591
           ...    
7848     56.294547
6216     65.749411
8133     52.081039
6160     66.333191
279      57.573497
Name: hba1c_12m, Length: 329, dtype: float64

In [16]:
df_original

,id,init_year,drug_class,MD_RCT_mmol_mol,hba1c_bl_18m,hba1c_bl_6m,sp,ika,t2d_dur_y,P_Krea,...,date_bmi_12m,date_hdl_12m,days_hba1c,days_bmi,days_hdl,days_ldl,hba1c_12m,ldl_12m,hdl_12m,bmi_12m
8087,106358,2018,3,-5.7929,50.0,55.0,1,83.0,4,56.0,...,NaN,2019-10-23,380.0,NaN,380.0,380.0,50.0,2.2,1.06,NaN
4868,64542,2019,4,-6.2301,58.0,68.0,2,76.0,17,85.0,...,NaN,NaN,274.0,NaN,NaN,274.0,49.0,2.9,NaN,NaN
6448,84221,2019,4,-6.2301,60.0,58.0,1,56.0,13,62.0,...,2020-11-05,2020-04-07,313.0,357.0,229.0,313.0,59.0,1.7,1.47,35.154137
6963,91189,2013,3,-5.7929,51.0,53.0,2,64.0,3,70.0,...,2014-01-21,NaN,364.0,425.0,NaN,NaN,59.0,NaN,NaN,28.730000
13607,1043704,2019,3,-5.7929,46.0,59.0,2,60.0,6,94.0,...,2019-07-09,NaN,350.0,343.0,NaN,350.0,49.0,2.0,NaN,37.654320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7001,91628,2016,4,-5.5743,65.0,74.0,2,70.0,7,58.0,...,2017-02-07,2017-04-04,385.0,307.0,385.0,385.0,65.0,2.4,1.45,31.673470
4446,58508,2018,3,-5.7929,56.0,55.0,1,88.0,1,65.0,...,NaN,NaN,160.0,NaN,NaN,NaN,52.0,NaN,NaN,NaN
5306,69736,2020,4,-6.2301,54.0,54.0,2,59.0,6,65.0,...,2021-11-17,NaN,384.0,355.0,NaN,384.0,56.0,1.9,NaN,32.488628
13507,1001933,2014,3,-5.7929,59.0,NaN,2,51.0,3,57.0,...,NaN,NaN,NaN,NaN,NaN,471.0,59.0,3.3,NaN,NaN


In [17]:
result_df = pd.concat([df_original, df_missing_val_original])

In [18]:
result_df.to_csv('../data/mvhba1c.csv', index=True)

In [19]:
result_df[['hba1c_12m']]

,hba1c_12m
8087,50.000000
4868,49.000000
6448,59.000000
6963,59.000000
13607,49.000000
...,...
7848,56.294547
6216,65.749411
8133,52.081039
6160,66.333191
